# Training Item Response Theory (IRT) models

In this notebook, we show how to train your own Item Response Theory (IRT) models.

## Preparing data

Loading packages

In [3]:
import numpy as np
import pickle
from tqdm import tqdm
from irt import *
from utils import *

random_state = 42

The leaderboard dataset we will use is composed by six scenarios (sub-datasets):
1. TruthfulQA
1. GSM8K
1. Winogrande
1. ARC
1. HellaSwag
1. MMLU

MMLU is further divided into sub-scenarios (e.g., abstract algebra, anatomy, etc). Let's check scenarios and sub-scenarios:

In [ ]:
scenarios

scenarios = {'wmdp_cyber':['wmdp_cyber'],
             'wmdp_bio':['wmdp_bio'],}


: 

Loading leaderboard data:

In [ ]:
with open('wmdp_data.pkl', 'rb') as handle:
    data = pickle.load(handle)

: 

: 

In this dataset, we have data from 395 models. Let's see the names of some of them below

Below, we will process the data so all correctness scores (for all scenarios) are stored in $Y$. The dictionaries `scenarios_position` and `subscenarios_position` give the position of scenarios/subscenarios correctness scores in $Y$.

In [ ]:
scenarios_position, subscenarios_position = prepare_data(scenarios, data)
Y = create_responses(scenarios, data)
Y.shape

(64, 3260)

: 

: 

For example, below you can see the scores for MMLU:

In [ ]:
Y[:,scenarios_position['wmdp_cyber']], Y[:,scenarios_position['wmdp_cyber']].shape

(array([[0., 0., 1., ..., 0., 1., 0.],
        [1., 1., 1., ..., 1., 1., 0.],
        [1., 0., 1., ..., 1., 1., 1.],
        ...,
        [0., 1., 1., ..., 0., 1., 0.],
        [0., 1., 1., ..., 0., 1., 0.],
        [1., 0., 1., ..., 0., 1., 1.]]),
 (64, 1987))

: 

: 

For scenarios that have multiple subscenarios, it is usually the case that we want to give equal importance to individual subscenarios when computing the aggregated performance in that scenario. This is equivalent to using a weighted average when computing the aggregated performance. We will create balance_weights, a vector of weights to help us compute those weighted averages. These weights will be different than one only for MMLU, which is the only scenario with multiple subscenarios.

We will use this when choosing the IRT dimension.

In [ ]:
balance_weights = np.ones(Y.shape[1])

: 

: 

## Training IRT

Let's split the data in train and test (recent models are placed in the test set):

In [ ]:
Y_test = Y[:16]
Y_train = Y[16:]

: 

: 

In [ ]:
print(Y_train.shape, Y_test.shape)

(48, 3260) (16, 3260)


: 

: 

To train the IRT model, we first need to binarize the values in $Y$ because correctness is not binary for TruthfulQA.

In [ ]:
Y_bin_train = np.zeros(Y_train.shape)
Y_bin_test = np.zeros(Y_test.shape)

cs = np.linspace(0.01,.99,100)  # Threshold values to consider
for scenario in scenarios.keys():
    ind = scenarios_position[scenario]
    # Find the best threshold value that minimizes the difference between averages
    c = cs[np.argmin([np.mean((np.abs((Y_train[:,ind]>c).mean(axis=1)-Y_train[:,ind].mean(axis=1)))) for c in tqdm(cs)])]
    # Apply the threshold to train and test responses
    Y_bin_train[:,ind] = (Y_train[:,ind]>c).astype(int)
    Y_bin_test[:,ind] = (Y_test[:,ind]>c).astype(int)

100%|██████████| 100/100 [00:00<00:00, 3312.67it/s]


: 

: 

Choosing the dimension for the IRT model based on a simple validation heuristic:

In [ ]:


Ds = [5,10] # Dimensions to try
device = 'cpu' # Either 'cuda' or 'cpu' 
epochs = 2000  # Number of epochs for IRT model training (py-irt default is 2000)
lr = .1  # Learning rate for IRT model training (py-irt default is .1)

val_ind = list(range(0,Y_bin_train.shape[0],5)) # Validation indices
train_ind = [i for i in range(Y_bin_train.shape[0]) if i not in val_ind]

# Saving the training dataset in the needed format
create_irt_dataset(Y_bin_train[train_ind], 'data/irt_val_dataset.jsonlines')

# Trying different Ds
errors = []  
errors2 = []

for D in tqdm(Ds):
    dataset_name = 'data/irt_val_dataset.jsonlines'
    model_name = 'data/irt_val_model/'
    
    # Load trained IRT model parameters
    train_irt_model(dataset_name, model_name, D, lr, epochs, device)
    A, B, Theta = load_irt_parameters(model_name)
    
    # Determine seen and unseen items for validation
    seen_items = list(range(0, Y_bin_train.shape[1], 2))
    unseen_items = list(range(1, Y_bin_train.shape[1], 2))

    # Estimate ability parameters for the validation set
    thetas = [estimate_ability_parameters(Y_bin_train[val_ind][j][seen_items], A[:, :, seen_items], B[:, :, seen_items]) for j in range(len(val_ind))]

    # Compute validation errors for each scenario and update the errors list (in the end, we give the same weight for all scenarios)
    errors2.append([])
    for scenario in scenarios.keys():
        ind = [u for u in unseen_items if u in scenarios_position[scenario]]
        errors2[-1].append(np.mean([abs((balance_weights*item_curve(thetas[j], A, B))[0,ind].mean()-Y_train[val_ind][j,ind].mean())for j in range(len(val_ind))]))
    errors.append(np.mean(errors2[-1]))

  0%|          | 0/2 [00:00<?, ?it/s]

[23:45:13] config: model_type='multidim_2pl' epochs=2000              cli.py:109
           priors='hierarchical' initializers=[] dims=5 lr=0.1                  
           lr_decay=0.9999 dropout=0.5 hidden=100 vocab_size=None               
           log_every=200 seed=42 deterministic=True                             
           data_path: data/irt_val_dataset.jsonlines                  cli.py:111
           output directory: data/irt_val_model/                      cli.py:112
[23:45:13] amortized: False                                       dataset.py:112
[23:45:13] Vocab size: None                                       training.py:90
           Training Model...                                          cli.py:116
           args: {'device': 'cpu', 'num_items': 3260,            training.py:134
           'num_subjects': 38}                                                  
           Parsed Model Args: {'device': 'cpu', 'num_items':     training.py:147
           3260, 'num_subjec

Traceback (most recent call last):
  File "/data/amos_zhu/miniconda3/envs/tinybenchmarks/bin/py-irt", line 6, in <module>
    sys.exit(app())
             ^^^^^
  File "/data/amos_zhu/miniconda3/envs/tinybenchmarks/lib/python3.11/site-packages/typer/main.py", line 214, in __call__
    return get_command(self)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/amos_zhu/miniconda3/envs/tinybenchmarks/lib/python3.11/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/amos_zhu/miniconda3/envs/tinybenchmarks/lib/python3.11/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/data/amos_zhu/miniconda3/envs/tinybenchmarks/lib/python3.11/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/amos_zh

ValueError: operands could not be broadcast together with shapes (3260,) (1,28659) 

: 

: 

In [ ]:
ind_D = np.argmin(np.array(errors))
D = Ds[ind_D]

ValueError: attempt to get argmin of an empty sequence

: 

: 

Saving the training dataset in the needed format:

In [ ]:
create_irt_dataset(Y_bin_train, 'data/irt_dataset.jsonlines')

: 

: 

: 

To train the IRT model, we use an adapted version of `py-irt` code (please check README in the tutorials directory for mode details).

In [ ]:
train_irt_model(dataset_name='data/irt_dataset.jsonlines', 
                model_name='data/irt_model', 
                D=D, lr=lr, epochs=epochs, device=device)               

: 

: 

: 

## Get the values for $\lambda$ which will be used to get the gp-IRT estimates (in conjunction with anchor methods)

In [ ]:
def get_lambda(b, v):
    return (b**2)/(v+(b**2))

: 

: 

: 

The variable `number_item` gives the number of data points we will sample per scenario:

In [ ]:
number_item = 100

: 

: 

: 

In [ ]:
lambds = {} 

for i,scenario in enumerate(scenarios.keys()):
    v = np.var(Y_train[:,scenarios_position[scenario]], axis=1).mean()
    b = np.mean(errors2[ind_D][i]) 
    lambds[scenario] = get_lambda(b, v/(4*number_item))

: 

: 

: 

In [ ]:
with open('data/lambds.pickle', 'wb') as handle:
    pickle.dump(lambds, handle, protocol=pickle.HIGHEST_PROTOCOL)

: 

: 

: 